In [1]:
import pandas as pd

In [6]:
# Load the CSV files
data_202406 = pd.read_csv('JC-202406-citibike-tripdata.csv')
data_202407 = pd.read_csv('JC-202407-citibike-tripdata.csv')
data_202408 = pd.read_csv('JC-202408-citibike-tripdata.csv')

In [7]:
# Combine the DataFrame into one
combined_data = pd.concat([data_202406, data_202407, data_202408], ignore_index=True)

In [12]:
# Print column names to verify
print("Columns in the DataFrame:", combined_data.columns)

# Check for missing values
print("Missing values:\n", combined_data.isnull().sum())

# Corrected column names based on your DataFrame
required_columns = ['started_at', 'ended_at', 'start_station_id', 'end_station_id']

# Check if all required columns are present in the DataFrame
for col in required_columns:
    if col not in combined_data.columns:
        print(f"Column '{col}' not found in the DataFrame")

# Drop rows with missing critical data (e.g., station or time information)
combined_data = combined_data.dropna(subset=required_columns)

# Convert date columns to datetime format using .loc to modify in place
combined_data.loc[:, 'started_at'] = pd.to_datetime(combined_data['started_at'])
combined_data.loc[:, 'ended_at'] = pd.to_datetime(combined_data['ended_at'])

# Add trip duration in minutes (if not present) using .loc to modify in place
if 'tripduration' not in combined_data.columns:
    combined_data.loc[:, 'tripduration'] = (combined_data['ended_at'] - combined_data['started_at']).dt.total_seconds() / 60

# Remove negative or zero trip durations
combined_data = combined_data[combined_data['tripduration'] > 0]

# Filter for realistic trip durations (e.g., between 1 and 180 minutes)
combined_data = combined_data[(combined_data['tripduration'] >= 1) & (combined_data['tripduration'] <= 180)]

# Check and remove duplicates
combined_data = combined_data.drop_duplicates()

# Save cleaned data to a new CSV file
combined_data.to_csv('Cleaned CitiBike Tripdata/cleaned_citibike_data.csv', index=False)

Columns in the DataFrame: Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'tripduration'],
      dtype='object')
Missing values:
 ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
tripduration          0
dtype: int64
